# Class : Genomic analysis on intervals & Genome Browsers

---
## Before Class
1. Install bedtools and pybedtools: `conda install pybedtools` - this installs both
* Review bedtools documentation ( https://bedtools.readthedocs.io/en/latest/ )
* Review pybedtools documentation ( https://daler.github.io/pybedtools/ )

---
## Learning Objectives

1. Performing operations on intervals using bedtools and pybedtools
* Use and understanding of the UCSC Genome Browser

---
## Background

In previous class, we identified eQTLs (expression quantitative trait loci) which are associated with gene expression levels. To further explore whether they are causative of the expression change and make hypothesis about the potential mechanisms, we can intersect the eQTLs with genomic features from other assays: e.g. DNase I hypersensitive site (DHS) which basically represents the open chromatin regions, chromHMM state which learns and characterizes chromatin states.

In today's class, we will first use *bedtools* and *pybedtools* to intersect eQTL variants with DHS and chromHMM state. Then we will pick one potentailly functional variant and view other genomic features around this variant through the UCSC genome browser.


---
## Operations on Intervals

We will be using eQTLs from the Genotype-Tissue Expression (GTEx) project (https://gtexportal.org) in today's class. We have converted the orginal text file to bed file format, and randomly selected 100 variants on chromosome 22 from liver tissue for simplicity.

Take a look at the variant file: `eQTL_variants.bed`

In [ ]:
!head data/eQTL_variants.bed

Column names: chromosome, start position, end position, reference allele, alternative allele, gene ID of associated gene 

We also provided other two bed files of genomic features we want to intersect with the variants:

DHS (DNase I hypersensitive site): `HepG2_DHS.bed`

In [ ]:
!head data/HepG2_DHS.bed

chromHMM state: `Liver_chromHMM.bed`

In [ ]:
!head data/Liver_chromHMM.bed

You can find the description of each chromatin state in last column from `Mnemonic_key.txt`

---
### bedtools
*bedtools* is a powerful toolset to perform operations on intervals, especially when you are working with large data such as human genome. 

`intersect` is the most commonly used command for *bedtools* (https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html), which compares two or more input files, and identifies all the overlapping regions.

<img src = "figures/bedtools_intersect.png">

In [ ]:
#Take a look at the usage information for bedtools intersect command.
#Note the input file formats and the different options for output.
! bedtools intersect --help

In [ ]:
#We can identify the variants overlapping DHS in our files with:
#bedtools -a <variant bed file> -b <DHS bed file>
! bedtools intersect -a data/eQTL_variants.bed -b data/HepG2_DHS.bed
#You should see four variants in output

In [ ]:
#Try adding other options to your command (e.g. -wb/-wo/-c), see how it will change your output:
#For example:
! bedtools intersect -a data/eQTL_variants.bed -b data/HepG2_DHS.bed -wb

---
### pybedtools
*pybedtools* is a python library which extends upon the *bedtools* suite with an intuitive python interface.


In [1]:
from pybedtools import BedTool

After importing *pybedtools*, we can create a `BedTool` object for each bed file:

In [2]:
variants = BedTool('data/eQTL_variants.bed')
DHS = BedTool('data/HepG2_DHS.bed')
chromHMM = BedTool('data/Liver_chromHMM.bed')

Then we can do intersection with the `intersect()` method: https://daler.github.io/pybedtools/intersections.html

#### Identify variants overlapping DHS

In [3]:
#Create a BedTool object variants_within_DHS here:
variants_within_DHS = variants.intersect(DHS)
print (variants_within_DHS)

chr22	29470114	29470115	A	G	ENSG00000183762.8
chr22	36861319	36861320	C	T	ENSG00000100350.10
chr22	42510982	42510983	C	G	ENSG00000227370.1
chr22	50675647	50675648	C	A	ENSG00000170638.5



We want to further find out the chromatin states of these variants overlapping DHS:

Hint: you can add options from *bedtools* by setting them as true in arg (e.g. `wb=True`);

In [4]:
#Print the chromatin states here:
print (variants_within_DHS.intersect(chromHMM,wb=True))

chr22	29470114	29470115	A	G	ENSG00000183762.8	chr22	29469400	29470400	2_TssAFlnk
chr22	36861319	36861320	C	T	ENSG00000100350.10	chr22	36861200	36862000	7_Enh
chr22	42510982	42510983	C	G	ENSG00000227370.1	chr22	42510600	42512200	9_Het
chr22	50675647	50675648	C	A	ENSG00000170638.5	chr22	50675400	50676000	6_EnhG



#### Identify variants overlapping enhancers

Now try identifying variants in enhancer regions (i.e. overlapping with regions labeled as "7_Enh" in `Liver_chromHMM.bed`)

Hint: check out the filter method: https://daler.github.io/pybedtools/filtering.html

In [5]:
##Create a new BedTool object variants_within_Enh here:
chromHMM_Enh = chromHMM.filter(lambda x: x[3] == '7_Enh')
variants_within_Enh = variants.intersect(chromHMM_Enh)
print (variants_within_Enh)

chr22	19814170	19814171	C	T	ENSG00000232926.1
chr22	24251516	24251517	C	T	ENSG00000099977.9
chr22	36861319	36861320	C	T	ENSG00000100350.10
chr22	42396149	42396150	G	A	ENSG00000213790.2
chr22	42538398	42538399	C	T	ENSG00000270083.1
chr22	46660667	46660668	C	T	ENSG00000075234.12
chr22	50966633	50966634	G	T	ENSG00000130489.8



#### Some (maybe) more challenging tasks:
1. How many variants are there in each chromatin state?

In [6]:
from collections import Counter
variants_chromHMM= variants.intersect(chromHMM,wb=True)
chromHMM_count = Counter()
for variant in variants_chromHMM:
    chromHMM_count[variant[9]] += 1
chromHMM_count

Counter({'7_Enh': 7,
         '4_Tx': 11,
         '15_Quies': 32,
         '1_TssA': 1,
         '14_ReprPCWk': 8,
         '9_Het': 4,
         '5_TxWk': 30,
         '6_EnhG': 5,
         '2_TssAFlnk': 2})

2. What's the average distance between the variants and their nearest exons? (Hint: check out the closest method, you will also need the gff file `Homo_sapiens.GRCh37.87.chromosome.22.gff3.gz`)

**Clarification**: the original gff file we provided has inconsistent naming convention with the bed file of variants (the first column was '22' instead of 'chr22'), which will cause errors for bedtools. The gff3 file has been updated, make sure you are using the latest version and sorry for the confusion.

In [7]:
genes = BedTool('data/Homo_sapiens.GRCh37.87.chromosome.22.gff3.gz')
exons = genes.filter(lambda x: x[2] == 'exon') #we only need annotations of exons

closest_exons = variants.closest(exons, d=True) #set d=True to output the distance

distances = []
for exon in closest_exons:
    distances.append(int(exon[-1]))  #the last column is the distance between variant and the closest exon 

average_distance = sum(distances)/len(distances)
print (average_distance)

1730.096256684492


---
## UCSC Genome Browser
Now let's pick the variant within DHS and in enhancer state: chr22:36861319-36861320, we can explore more genomic features around this variant through UCSC genome browser. You can find some helpful screenshots under `figures/UCSC_genome_browser.pdf`.
1. Go to https://genome.ucsc.edu/, choose Human GRCh37/hg19 build.
* Search for the varaint chr22:36861319-36861320.
* Explore information from different tracks, for examples:
  * ENCODE Regulation -> What are the ChIP-seq peaks around this variant?
  * Conservation -> What does the conservation score imply?